# <center>Few-shot Object Detection via Feature Reweighting </center>

In this notebook we will explore and discuss a few-shot object detection learning method called "Few-shot Object Detection via Feature Reweighting". The proposed method is built on two main modules, meta feature learner and reweighting module. The method leverages fully labeled base classes and adapts the model to novel classes using a meta feature learner and a reweighting module within a one-stage detection architecture. Indeed, given some base classes containing enough examples and some novel classes containing few examples, the proposed method aims at obtaining a model that can detect both base and novel objects at test time. In more details, using base classes with enough samples, the feature learner extracts meta features that are generalizable to detect novel object classes, and the reweighting module maps a few support examples obtained from novel classes to vectors that indicate for each meta feature the importance to detect a corresponding object. Note that the feature learner, the reweighting module and a detection prediction module are all trained together end-to-end based on an few-shot learning scenario.

In this notebook, we use Pytorch to code and test the proposed method.

## Feature Reweighting for Detection
The proposed few shot object detection method introduces a meta feature learner D and a reweighting module M into a one-stage detection framework. Note that YOLOv2 (i.e. DarkNet-19) is adapted to implement the meta feature extractor D. To implement the reweighting module M, a light-weight CNN is designed to ease its learning and enhance efficiency. 

The meta feature learner D learns how to extract meta features for given input query images in order to detect their novel objects. The reweighting module M uses support samples as input, learns how to embed inputs into reweighting vectors, and uses these vectors to adjust the contribution of each meta feature of the query image and then gives adjusted meta features to detection prediction module as described in the figure bellow.

<img src="Screenshot from 2020-02-06 09-55-46.png">

Note that in object detection task, one image may contain more than one multiple objects from different classes. Consequently, in addition to RGB image, an additional mask is included to mask all objects but one, and thus lets the reweighting module knows what the target class is, as shown in the figure above.

## Learning Scheme

To ensure the model generalization performance when using only few examples, the proposed method aims at developing a new two-phase learning scheme that is different from the classical ones when learning a detection model.


The first phase is called the "base training phase", in which the feature learner, detection prediction module and reweighting module are trained together on base classes. This is to make them coordinate since the model have to learn how to detect objects of interest by referring to a good rewieghting vector.


The second phase is called "few-shot fine-tuning" in which the model is trained on both base and novel classes. Note that the training procedure is the same as the first except that it requires only few iterations for the model to converge. 

In both training phases, the reweighting module parameters depends on the input pairs : support image, and mask (or bounding box) that are randomly sampled from available data at each iteration. Note that after the second phase, we would like to obtain a detection model that can directly perform detection without requiring any support input. This can be done by setting the reweighting vector for a target class to the average one predicted by the model after taking the k-shot samples as input. After this, the reweighting module can be completely removed during inference. Therefore, our model adds negligible extra model parameters to the original detector.

## Few-shot Object Detection via Feature Reweighting Implementation

To implement the proposed method, the two training phases are considered. For each training phase we evaluate the model and get its performance. The proposed method is evaluated using VOC2007 test set for testing, and VOC2007 and VOC2012 train/val sets for training. Note that VOC contains 20 object categories, and in our implementation we randomly select 5 classes as novel ones and keep the remaining 15 ones as the base classes.  



## Prepare dataset

Before training, data need to be prepared. To do so we start by obtained the Pascal VOC Data as follows:

* cd data

* wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar

* wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar

* wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar

* tar xf VOCtrainval_11-May-2012.tar

* tar xf VOCtrainval_06-Nov-2007.tar

* tar xf VOCtest_06-Nov-2007.tar

Now we need to labels for VOC as follows:
* wget http://pjreddie.com/media/files/voc_label.py
* python voc_label.py
* cat 2007_train.txt 2007_val.txt 2012_*.txt > voc_train.txt

Then, we generate per-class labels for VOC (used for meta inpput) as follows:
* cp ../scripts/voc_label_1c.py .
* python voc_label_1c.py

And finally, we generate few-shot image list to use as our few-shot datasets as follows:
* cd ..
* python scripts/convert_fewlist.py 

## Phase one: base training phase

This is the first learning phase in which the feature learner, detection predection module and reweighting module are trained together on base classes. 

In [1]:
from __future__ import print_function

import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable

import dataset
import random
import math
import os
from utils import *
from cfg import parse_cfg, cfg
from darknet_meta import Darknet
from models.tiny_yolo import TinyYoloNet
import pdb

# Training settings
datacfg       = 'cfg/metayolo.data'
darknetcfg    = parse_cfg('cfg/darknet_dynamic.cfg')
learnetcfg    = parse_cfg('cfg/reweighting_net.cfg')
weightfile    = 'darknet19_448.conv.23'

data_options  = read_data_cfg(datacfg)
net_options   = darknetcfg[0]
meta_options  = learnetcfg[0]

# Configure options
cfg.config_data(data_options)
cfg.config_meta(meta_options)
cfg.config_net(net_options)

# Parameters 
metadict      = data_options['meta']
trainlist     = data_options['train']

testlist      = data_options['valid']
backupdir     = data_options['backup']
gpus          = data_options['gpus']  # e.g. 0,1,2,3
ngpus         = len(gpus.split(','))
num_workers   = int(data_options['num_workers'])

batch_size    = int(net_options['batch'])
max_batches   = int(net_options['max_batches'])
learning_rate = float(net_options['learning_rate'])
momentum      = float(net_options['momentum'])
decay         = float(net_options['decay'])
steps         = [float(step) for step in net_options['steps'].split(',')]
scales        = [float(scale) for scale in net_options['scales'].split(',')]

#Train parameters
use_cuda      = True
seed          = int(time.time())
eps           = 1e-5
dot_interval  = 70  # batches
# save_interval = 10  # epoches

# Test parameters
conf_thresh   = 0.25
nms_thresh    = 0.4
iou_thresh    = 0.5

## --------------------------------------------------------------------------
## MAIN
backupdir = cfg.backup
print('logging to ' + backupdir)
if not os.path.exists(backupdir):
    os.mkdir(backupdir)

torch.manual_seed(seed)
#if use_cuda:
#    os.environ['CUDA_VISIBLE_DEVICES'] = gpus
#    torch.cuda.manual_seed(seed)

model       = Darknet(darknetcfg, learnetcfg)
region_loss = model.loss

model.load_weights(weightfile)
model.print_network()


###################################################
### Meta-model parameters
region_loss.seen  = model.seen
processed_batches = 0 if cfg.tuning else model.seen/batch_size
trainlist         = dataset.build_dataset(data_options)
nsamples          = len(trainlist)
init_width        = model.width
init_height       = model.height
init_epoch        = 0 if cfg.tuning else model.seen/nsamples
max_epochs        = max_batches*batch_size/nsamples+1
max_epochs        = int(math.ceil(cfg.max_epoch*1./cfg.repeat)) if cfg.tuning else max_epochs 
print(cfg.repeat, nsamples, max_batches, batch_size)
print(num_workers)

kwargs = {'num_workers': num_workers, 'pin_memory': True} if use_cuda else {}
test_loader = torch.utils.data.DataLoader(
    dataset.listDataset(testlist, shape=(init_width, init_height),
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]), train=False),
    batch_size=batch_size, shuffle=False, **kwargs)

#print(metadict)
test_metaset = dataset.MetaDataset(metafiles=metadict, train=True)
test_metaloader = torch.utils.data.DataLoader(
    test_metaset,
    batch_size=test_metaset.batch_size,
    shuffle=False,
    num_workers=num_workers//2,
    pin_memory=True
)

# Adjust learning rate
factor = len(test_metaset.classes)
if cfg.neg_ratio == 'full':
    factor = 15.
elif cfg.neg_ratio == 1:
    factor = 3.0
elif cfg.neg_ratio == 0:
    factor = 1.5
elif cfg.neg_ratio == 5:
    factor = 8.0

print('factor:', factor)
learning_rate /= factor

if use_cuda:
    if ngpus > 1:
        model = torch.nn.DataParallel(model).cuda()
    else:
        model = model.cuda()

optimizer = optim.SGD(model.parameters(),
                      lr=learning_rate/batch_size,
                      momentum=momentum,
                      dampening=0,
                      weight_decay=decay*batch_size*factor)


def adjust_learning_rate(optimizer, batch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = learning_rate
    for i in range(len(steps)):
        scale = scales[i] if i < len(scales) else 1
        if batch >= steps[i]:
            lr = lr * scale
            if batch == steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr/batch_size
    return lr

def train(epoch):
    global processed_batches
    t0 = time.time()
    if ngpus > 1:
        cur_model = model.module
    else:
        cur_model = model

    train_loader = torch.utils.data.DataLoader(
        dataset.listDataset(trainlist, shape=(init_width, init_height),
                       shuffle=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                       ]), 
                       train=True, 
                       seen=cur_model.seen,
                       batch_size=batch_size,
                       num_workers=num_workers),
        batch_size=batch_size, shuffle=False, **kwargs)

    metaset = dataset.MetaDataset(metafiles=metadict, train=True)
    metaloader = torch.utils.data.DataLoader(
        metaset,
        batch_size=metaset.batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    metaloader = iter(metaloader)

    lr = adjust_learning_rate(optimizer, processed_batches)
    logging('epoch %d/%d, processed %d samples, lr %f' % (epoch, max_epochs, epoch * len(train_loader.dataset), lr))

    model.train()
    t1 = time.time()
    avg_time = torch.zeros(9)
    for batch_idx, (data, target) in enumerate(train_loader):
        metax, mask = metaloader.next()
        t2 = time.time()
        adjust_learning_rate(optimizer, processed_batches)
        processed_batches = processed_batches + 1

        if use_cuda:
            data = data.cuda()
            metax = metax.cuda()
            mask = mask.cuda()
            #target= target.cuda()
        t3 = time.time()
        data, target = Variable(data), Variable(target)
        metax, mask = Variable(metax), Variable(mask)
        t4 = time.time()
        optimizer.zero_grad()
        t5 = time.time()
        output = model(data, metax, mask)
        t6 = time.time()
        region_loss.seen = region_loss.seen + data.data.size(0)
        loss = region_loss(output, target)
        t7 = time.time()
        loss.backward()
        del loss
        torch.cuda.empty_cache()
        t8 = time.time()
        optimizer.step()
        t9 = time.time()
        if False and batch_idx > 1:
            avg_time[0] = avg_time[0] + (t2-t1)
            avg_time[1] = avg_time[1] + (t3-t2)
            avg_time[2] = avg_time[2] + (t4-t3)
            avg_time[3] = avg_time[3] + (t5-t4)
            avg_time[4] = avg_time[4] + (t6-t5)
            avg_time[5] = avg_time[5] + (t7-t6)
            avg_time[6] = avg_time[6] + (t8-t7)
            avg_time[7] = avg_time[7] + (t9-t8)
            avg_time[8] = avg_time[8] + (t9-t1)
            print('-------------------------------')
            print('       load data : %f' % (avg_time[0]/(batch_idx)))
            print('     cpu to cuda : %f' % (avg_time[1]/(batch_idx)))
            print('cuda to variable : %f' % (avg_time[2]/(batch_idx)))
            print('       zero_grad : %f' % (avg_time[3]/(batch_idx)))
            print(' forward feature : %f' % (avg_time[4]/(batch_idx)))
            print('    forward loss : %f' % (avg_time[5]/(batch_idx)))
            print('        backward : %f' % (avg_time[6]/(batch_idx)))
            print('            step : %f' % (avg_time[7]/(batch_idx)))
            print('           total : %f' % (avg_time[8]/(batch_idx)))
        t1 = time.time()
    print('')
    t1 = time.time()
    logging('training with %f samples/s' % (len(train_loader.dataset)/(t1-t0)))

    if (epoch+1) % cfg.save_interval == 0:
        logging('save weights to %s/%06d.weights' % (backupdir, epoch+1))
        cur_model.seen = (epoch + 1) * len(train_loader.dataset)
        cur_model.save_weights('%s/%06d.weights' % (backupdir, epoch+1))


def test(epoch):
    def truths_length(truths):
        for i in range(50):
            if truths[i][1] == 0:
                return i

    model.eval()
    if ngpus > 1:
        cur_model = model.module
    else:
        cur_model = model
    num_classes = cur_model.num_classes
    anchors     = cur_model.anchors
    num_anchors = cur_model.num_anchors
    total       = 0.0
    proposals   = 0.0
    correct     = 0.0

    _test_metaloader = iter(test_metaloader)
    for batch_idx, (data, target) in enumerate(test_loader):
        metax, mask = _test_metaloader.next()
        if use_cuda:
            data = data.cuda()
            metax = metax.cuda()
            mask = mask.cuda()
        data = Variable(data, volatile=True)
        metax = Variable(metax, volatile=True)
        mask = Variable(mask, volatile=True)
        output = model(data, metax, mask).data
        all_boxes = get_region_boxes(output, conf_thresh, num_classes, anchors, num_anchors)
        for i in range(output.size(0)):
            boxes = all_boxes[i]
            boxes = nms(boxes, nms_thresh)
            truths = target[i].view(-1, 5)
            num_gts = truths_length(truths)
     
            total = total + num_gts
    
            for i in range(len(boxes)):
                if boxes[i][4] > conf_thresh:
                    proposals = proposals+1

            for i in range(num_gts):
                box_gt = [truths[i][1], truths[i][2], truths[i][3], truths[i][4], 1.0, 1.0, truths[i][0]]
                best_iou = 0
                best_j = -1
                for j in range(len(boxes)):
                    iou = bbox_iou(box_gt, boxes[j], x1y1x2y2=False)
                    if iou > best_iou:
                        best_j = j
                        best_iou = iou
                if best_iou > iou_thresh and boxes[best_j][6] == box_gt[6]:
                    correct = correct+1

    precision = 1.0*correct/(proposals+eps)
    recall = 1.0*correct/(total+eps)
    fscore = 2.0*precision*recall/(precision+recall+eps)
    logging("precision: %f, recall: %f, fscore: %f" % (precision, recall, fscore))



evaluate = False
if evaluate:
    logging('evaluating ...')
    test(0)
else:
    for epoch in range(init_epoch, max_epochs):
        train(epoch)
        # test(epoch)

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/coco.names
('save_interval', 10)
['bird', 'bus', 'cow', 'motorbike', 'sofa']
('base_ids', [0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 18, 19])
logging to backup/metayolo_novel0_neg1
('class_scale', 1)
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
    9 conv    128  1 x 1 / 1    52 x  5

/network/home/bouklihg/power9/anaconda3/envs/p9_p2.7/lib/python2.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/network/home/bouklihg/power9/anaconda3/envs/p9_p2.7/lib/python2.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


64: nGT 156, recall 64, proposals 152699, loss: x 276.651245, y 362.598480, w 4985.898438, h 6742.632812, conf 19926.310547, cls 409.156494, total 32703.248047
128: nGT 145, recall 62, proposals 151682, loss: x 252.375793, y 334.918701, w 5796.765625, h 6346.075684, conf 17927.396484, cls 376.759216, total 31034.291016
192: nGT 176, recall 76, proposals 159585, loss: x 227.353607, y 274.520325, w 3573.720459, h 5902.065430, conf 15670.241211, cls 463.100647, total 26111.003906
256: nGT 163, recall 77, proposals 170976, loss: x 214.148254, y 261.624420, w 4083.257080, h 4140.985840, conf 13100.931641, cls 433.685822, total 22234.632812
320: nGT 145, recall 62, proposals 141394, loss: x 153.850037, y 183.730743, w 1799.773315, h 2615.840576, conf 8636.466797, cls 391.096802, total 13780.757812
384: nGT 177, recall 86, proposals 115850, loss: x 139.138397, y 149.674133, w 1980.108643, h 1891.314209, conf 6806.794434, cls 469.617096, total 11436.646484


KeyboardInterrupt: 

Note that since the training step above requires a large memory, computational power and time, the trained model on only base classes can be downloadded on: https://drive.google.com/file/d/1CSVFhfOHmRlbUsMu_eyBCvBWn_06a9zH/view

## Second phase: Few-shot Tuning

This is the second learning phase which the model is trained on both base and novel classes. Since this step requires only few iterations for the model to converge, we use only 10 epochs instead of 353 epochs required in first phase. Note that in few-shot fine-tuning, we use a very small set of training images to ensure that each class of objects has only k annotated bounding boxes. In our experiment, we choose k=5.

In [3]:
from __future__ import print_function
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable

import dataset
import random
import math
import os
from utils import *
from cfg import parse_cfg, cfg
from darknet_meta import Darknet
from models.tiny_yolo import TinyYoloNet
import pdb

# Training settings
datacfg       = "cfg/metatune.data"
darknetcfg    = parse_cfg("cfg/darknet_dynamic.cfg")
learnetcfg    = parse_cfg("cfg/reweighting_net.cfg")
weightfile    = "backup/metayolo_novel0_neg1/000150.weights"                          

data_options  = read_data_cfg(datacfg)
net_options   = darknetcfg[0]
meta_options  = learnetcfg[0]

# Configure options
cfg.config_data(data_options)
cfg.config_meta(meta_options)
cfg.config_net(net_options)

# Parameters 
metadict      = data_options['meta']
trainlist     = data_options['train']

testlist      = data_options['valid']
backupdir     = data_options['backup']
gpus          = data_options['gpus']  # e.g. 0,1,2,3
ngpus         = len(gpus.split(','))
num_workers   = int(data_options['num_workers'])

batch_size    = int(net_options['batch'])
max_batches   = int(net_options['max_batches'])
learning_rate = float(net_options['learning_rate'])
momentum      = float(net_options['momentum'])
decay         = float(net_options['decay'])
steps         = [float(step) for step in net_options['steps'].split(',')]
scales        = [float(scale) for scale in net_options['scales'].split(',')]

#Train parameters
use_cuda      = True
seed          = int(time.time())
eps           = 1e-5
dot_interval  = 70  # batches
# save_interval = 10  # epoches

# Test parameters
conf_thresh   = 0.25
nms_thresh    = 0.4
iou_thresh    = 0.5

## --------------------------------------------------------------------------
## MAIN
backupdir = cfg.backup
print('logging to ' + backupdir)
if not os.path.exists(backupdir):
    os.mkdir(backupdir)

torch.manual_seed(seed)
if use_cuda:
    os.environ['CUDA_VISIBLE_DEVICES'] = gpus
    torch.cuda.manual_seed(seed)

model       = Darknet(darknetcfg, learnetcfg)
region_loss = model.loss

model.load_weights(weightfile)
model.print_network()


###################################################
### Meta-model parameters
region_loss.seen  = model.seen
processed_batches = 0 if cfg.tuning else model.seen/batch_size
trainlist         = dataset.build_dataset(data_options)
nsamples          = len(trainlist)
init_width        = model.width
init_height       = model.height
init_epoch        = 0 if cfg.tuning else model.seen/nsamples
max_epochs        = max_batches*batch_size/nsamples+1
max_epochs        = int(math.ceil(cfg.max_epoch*1./cfg.repeat)) if cfg.tuning else max_epochs 
print(cfg.repeat, nsamples, max_batches, batch_size)
print(num_workers)

kwargs = {'num_workers': num_workers, 'pin_memory': True} if use_cuda else {}
test_loader = torch.utils.data.DataLoader(
    dataset.listDataset(testlist, shape=(init_width, init_height),
                   shuffle=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]), train=False),
    batch_size=batch_size, shuffle=False, **kwargs)

test_metaset = dataset.MetaDataset(metafiles=metadict, train=True)
test_metaloader = torch.utils.data.DataLoader(
    test_metaset,
    batch_size=test_metaset.batch_size,
    shuffle=False,
    num_workers=num_workers//2,
    pin_memory=True
)

# Adjust learning rate
factor = len(test_metaset.classes)
if cfg.neg_ratio == 'full':
    factor = 15.
elif cfg.neg_ratio == 1:
    factor = 3.0
elif cfg.neg_ratio == 0:
    factor = 1.5
elif cfg.neg_ratio == 5:
    factor = 8.0

print('factor:', factor)
learning_rate /= factor

if use_cuda:
    if ngpus > 1:
        model = torch.nn.DataParallel(model).cuda()
    else:
        model = model.cuda()

optimizer = optim.SGD(model.parameters(),
                      lr=learning_rate/batch_size,
                      momentum=momentum,
                      dampening=0,
                      weight_decay=decay*batch_size*factor)


def adjust_learning_rate(optimizer, batch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = learning_rate
    for i in range(len(steps)):
        scale = scales[i] if i < len(scales) else 1
        if batch >= steps[i]:
            lr = lr * scale
            if batch == steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr/batch_size
    return lr

def train(epoch):
    global processed_batches
    t0 = time.time()
    if ngpus > 1:
        cur_model = model.module
    else:
        cur_model = model

    train_loader = torch.utils.data.DataLoader(
        dataset.listDataset(trainlist, shape=(init_width, init_height),
                       shuffle=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                       ]), 
                       train=True, 
                       seen=cur_model.seen,
                       batch_size=batch_size,
                       num_workers=num_workers),
        batch_size=batch_size, shuffle=False, **kwargs)

    metaset = dataset.MetaDataset(metafiles=metadict, train=True)
    metaloader = torch.utils.data.DataLoader(
        metaset,
        batch_size=metaset.batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    metaloader = iter(metaloader)

    lr = adjust_learning_rate(optimizer, processed_batches)
    logging('epoch %d/%d, processed %d samples, lr %f' % (epoch, max_epochs, epoch * len(train_loader.dataset), lr))

    model.train()
    t1 = time.time()
    avg_time = torch.zeros(9)
    for batch_idx, (data, target) in enumerate(train_loader):
        metax, mask = metaloader.next()
        t2 = time.time()
        adjust_learning_rate(optimizer, processed_batches)
        processed_batches = processed_batches + 1

        if use_cuda:
            data = data.cuda()
            metax = metax.cuda()
            mask = mask.cuda()
            #target= target.cuda()
        t3 = time.time()
        data, target = Variable(data), Variable(target)
        metax, mask = Variable(metax), Variable(mask)
        t4 = time.time()
        optimizer.zero_grad()
        t5 = time.time()
        output = model(data, metax, mask)
        t6 = time.time()
        region_loss.seen = region_loss.seen + data.data.size(0)
        loss = region_loss(output, target)
        t7 = time.time()
        loss.backward()
        t8 = time.time()
        optimizer.step()
        t9 = time.time()
        if False and batch_idx > 1:
            avg_time[0] = avg_time[0] + (t2-t1)
            avg_time[1] = avg_time[1] + (t3-t2)
            avg_time[2] = avg_time[2] + (t4-t3)
            avg_time[3] = avg_time[3] + (t5-t4)
            avg_time[4] = avg_time[4] + (t6-t5)
            avg_time[5] = avg_time[5] + (t7-t6)
            avg_time[6] = avg_time[6] + (t8-t7)
            avg_time[7] = avg_time[7] + (t9-t8)
            avg_time[8] = avg_time[8] + (t9-t1)
            print('-------------------------------')
            print('       load data : %f' % (avg_time[0]/(batch_idx)))
            print('     cpu to cuda : %f' % (avg_time[1]/(batch_idx)))
            print('cuda to variable : %f' % (avg_time[2]/(batch_idx)))
            print('       zero_grad : %f' % (avg_time[3]/(batch_idx)))
            print(' forward feature : %f' % (avg_time[4]/(batch_idx)))
            print('    forward loss : %f' % (avg_time[5]/(batch_idx)))
            print('        backward : %f' % (avg_time[6]/(batch_idx)))
            print('            step : %f' % (avg_time[7]/(batch_idx)))
            print('           total : %f' % (avg_time[8]/(batch_idx)))
        t1 = time.time()
    print('')
    t1 = time.time()
    logging('training with %f samples/s' % (len(train_loader.dataset)/(t1-t0)))

    if (epoch+1) % cfg.save_interval == 0:
        logging('save weights to %s/%06d.weights' % (backupdir, epoch+1))
        cur_model.seen = (epoch + 1) * len(train_loader.dataset)
        cur_model.save_weights('%s/%06d.weights' % (backupdir, epoch+1))


def test(epoch):
    def truths_length(truths):
        for i in range(50):
            if truths[i][1] == 0:
                return i

    model.eval()
    if ngpus > 1:
        cur_model = model.module
    else:
        cur_model = model
    num_classes = cur_model.num_classes
    anchors     = cur_model.anchors
    num_anchors = cur_model.num_anchors
    total       = 0.0
    proposals   = 0.0
    correct     = 0.0

    _test_metaloader = iter(test_metaloader)
    for batch_idx, (data, target) in enumerate(test_loader):
        metax, mask = _test_metaloader.next()
        if use_cuda:
            data = data.cuda()
            metax = metax.cuda()
            mask = mask.cuda()
        data = Variable(data, volatile=True)
        metax = Variable(metax, volatile=True)
        mask = Variable(mask, volatile=True)
        output = model(data, metax, mask).data
        all_boxes = get_region_boxes(output, conf_thresh, num_classes, anchors, num_anchors)
        for i in range(output.size(0)):
            boxes = all_boxes[i]
            boxes = nms(boxes, nms_thresh)
            truths = target[i].view(-1, 5)
            num_gts = truths_length(truths)
     
            total = total + num_gts
    
            for i in range(len(boxes)):
                if boxes[i][4] > conf_thresh:
                    proposals = proposals+1

            for i in range(num_gts):
                box_gt = [truths[i][1], truths[i][2], truths[i][3], truths[i][4], 1.0, 1.0, truths[i][0]]
                best_iou = 0
                best_j = -1
                for j in range(len(boxes)):
                    iou = bbox_iou(box_gt, boxes[j], x1y1x2y2=False)
                    if iou > best_iou:
                        best_j = j
                        best_iou = iou
                if best_iou > iou_thresh and boxes[best_j][6] == box_gt[6]:
                    correct = correct+1

    precision = 1.0*correct/(proposals+eps)
    recall = 1.0*correct/(total+eps)
    fscore = 2.0*precision*recall/(precision+recall+eps)
    logging("precision: %f, recall: %f, fscore: %f" % (precision, recall, fscore))



evaluate = False
if evaluate:
    logging('evaluating ...')
    test(0)
else:
    for epoch in range(init_epoch, max_epochs):
        train(epoch)
        # test(epoch)

('save_interval', 1)
['bird', 'bus', 'cow', 'motorbike', 'sofa']
('base_ids', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
logging to backup/metatunetest1_novel0_neg0
('class_scale', 1)
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
    9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128
   10 conv    256  3

KeyboardInterrupt: 

## Evaluate the Model

After training the model, now we will evaluate its performance. To do so we will consider the mean Average Precision (mPA), a metric that is usually used to measure the performance of methods in object detection field.

In [6]:
from darknet_meta import Darknet
import dataset
import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from utils import *
from cfg import cfg
from cfg import parse_cfg
import os
import pdb


def valid(datacfg, darknetcfg, learnetcfg, weightfile, outfile, use_baserw=False):
    options = read_data_cfg(datacfg)
    valid_images = options['valid']
    metadict = options['meta']
    # name_list = options['names']
    # backup = cfg.backup
    ckpt = weightfile.split('/')[-1].split('.')[0]
    backup = weightfile.split('/')[-2]
    ckpt_pre = '/ene_' if use_baserw else '/ene'
    prefix = 'results/' + backup.split('/')[-1] + ckpt_pre + ckpt
    print('saving to: ' + prefix)
    # prefix = 'results/' + weightfile.split('/')[1]
    # names = load_class_names(name_list)

    with open(valid_images) as fp:
        tmp_files = fp.readlines()
        valid_files = [item.rstrip() for item in tmp_files]
    
    m = Darknet(darknetcfg, learnetcfg)
    m.print_network()
    m.load_weights(weightfile)
    m.cuda()
    m.eval()

    valid_dataset = dataset.listDataset(valid_images, shape=(m.width, m.height),
                       shuffle=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                       ]))
    valid_batchsize = 2
    assert(valid_batchsize > 1)

    kwargs = {'num_workers': 4, 'pin_memory': True}
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=valid_batchsize, shuffle=False, **kwargs) 


    if False:
        metaset = dataset.MetaDataset(metafiles=metadict, train=False, ensemble=True)
        metaloader = torch.utils.data.DataLoader(
            metaset,
            batch_size=len(metaset),
            shuffle=False,
            **kwargs
        )
        metaloader = iter(metaloader)
        n_cls = len(metaset.classes)

        print('===> Generating dynamic weights...')
        metax, mask = metaloader.next()
        metax, mask = metax.cuda(), mask.cuda()
        metax, mask = Variable(metax, volatile=True), Variable(mask, volatile=True)
        dynamic_weights = m.meta_forward(metax, mask)

        for i in range(len(dynamic_weights)):
            assert dynamic_weights[i].size(0) == sum(metaset.meta_cnts)
            inds = np.cumsum([0] + metaset.meta_cnts)
            new_weight = []
            for j in range(len(metaset.meta_cnts)):
                new_weight.append(torch.mean(dynamic_weights[i][inds[j]:inds[j+1]], dim=0))
            dynamic_weights[i] = torch.stack(new_weight)
            print(dynamic_weights[i].shape)
    else:
        metaset = dataset.MetaDataset(metafiles=metadict, train=False, ensemble=True, with_ids=True)
        metaloader = torch.utils.data.DataLoader(
            metaset,
            batch_size=64,
            shuffle=False,
            **kwargs
        )
        # metaloader = iter(metaloader)
        n_cls = len(metaset.classes)

        enews = [0.0] * n_cls
        cnt = [0.0] * n_cls
        print('===> Generating dynamic weights...')
        kkk = 0
        for metax, mask, clsids in metaloader:
            print('===> {}/{}'.format(kkk, len(metaset) // 64))
            kkk += 1
            metax, mask = metax.cuda(), mask.cuda()
            metax, mask = Variable(metax, volatile=True), Variable(mask, volatile=True)
            dws = m.meta_forward(metax, mask)
            dw = dws[0]
            for ci, c in enumerate(clsids):
                enews[c] = enews[c] * cnt[c] / (cnt[c] + 1) + dw[ci] / (cnt[c] + 1)
                cnt[c] += 1
        dynamic_weights = [torch.stack(enews)]

        # import pickle
        # with open('data/rws/voc_novel2_.pkl', 'wb') as f:
        #     tmp = [x.data.cpu().numpy() for x in dynamic_weights]
        #     pickle.dump(tmp, f)
        # import pdb; pdb.set_trace()

        if use_baserw:
            import pickle
            # f = 'data/rws/voc_novel{}_.pkl'.format(cfg.novelid)
            f = 'data/rws/voc_novel{}_.pkl'.format(0)
            print('===> Loading from {}...'.format(f))
            with open(f, 'rb') as f:
            # with open('data/rws/voc_novel0_.pkl', 'rb') as f:
                rws = pickle.load(f)
                rws = [Variable(torch.from_numpy(rw)).cuda() for rw in rws]
                tki = cfg._real_base_ids
                for i in range(len(rws)):
                    dynamic_weights[i][tki] = rws[i][tki]
                    # dynamic_weights[i] = rws[i]
            # pdb.set_trace()


    if not os.path.exists(prefix):
        # os.mkdir(prefix)
        os.makedirs(prefix)

    fps = [0]*n_cls
    for i, cls_name in enumerate(metaset.classes):
        buf = '%s/%s%s.txt' % (prefix, outfile, cls_name)
        fps[i] = open(buf, 'w')
   
    lineId = -1
    
    conf_thresh = 0.005
    nms_thresh = 0.45
    for batch_idx, (data, target) in enumerate(valid_loader):
        data = data.cuda()
        data = Variable(data, volatile = True)
        output = m.detect_forward(data, dynamic_weights)

        if isinstance(output, tuple):
            output = (output[0].data, output[1].data)
        else:
            output = output.data

        # import pdb; pdb.set_trace()
        batch_boxes = get_region_boxes_v2(output, n_cls, conf_thresh, m.num_classes, m.anchors, m.num_anchors, 0, 1)

        if isinstance(output, tuple):
            bs = output[0].size(0)
        else:
            assert output.size(0) % n_cls == 0
            bs = output.size(0) // n_cls

        for b in range(bs):
            lineId = lineId + 1
            imgpath = valid_dataset.lines[lineId].rstrip()
            print(imgpath)
            imgid = os.path.basename(imgpath).split('.')[0]
            width, height = get_image_size(imgpath)
            for i in range(n_cls):
                # oi = i * bs + b
                oi = b * n_cls + i
                boxes = batch_boxes[oi]
                boxes = nms(boxes, nms_thresh)
                for box in boxes:
                    x1 = (box[0] - box[2]/2.0) * width
                    y1 = (box[1] - box[3]/2.0) * height
                    x2 = (box[0] + box[2]/2.0) * width
                    y2 = (box[1] + box[3]/2.0) * height

                    det_conf = box[4]
                    for j in range((len(box)-5)/2):
                        cls_conf = box[5+2*j]
                        cls_id = box[6+2*j]
                        prob =det_conf * cls_conf
                        fps[i].write('%s %f %f %f %f %f\n' % (imgid, prob, x1, y1, x2, y2))

    for i in range(n_cls):
        fps[i].close()

    # import pdb; pdb.set_trace()


datacfg = "cfg/metatune.data"
darknet = parse_cfg("cfg/darknet_dynamic.cfg")
learnet = parse_cfg("cfg/reweighting_net.cfg")
weightfile = "backup/metatunetest1_novel0_neg0/000005.weights"
gpu = '0'
use_baserw = False

data_options  = read_data_cfg(datacfg)
net_options   = darknet[0]
meta_options  = learnet[0]
data_options['gpus'] = gpu
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

# Configure options
cfg.config_data(data_options)
cfg.config_meta(meta_options)
cfg.config_net(net_options)

outfile = 'comp4_det_test_'
valid(datacfg, darknet, learnet, weightfile, outfile, use_baserw)


('save_interval', 1)
['bird', 'bus', 'cow', 'motorbike', 'sofa']
('base_ids', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
saving to: results/metatunetest1_novel0_neg0/ene000005
('class_scale', 1)
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256
    9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128
   10 con

/network/home/bouklihg/power9/anaconda3/envs/p9_p2.7/lib/python2.7/site-packages/ipykernel_launcher.py:94: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/network/home/bouklihg/power9/anaconda3/envs/p9_p2.7/lib/python2.7/site-packages/ipykernel_launcher.py:139: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000001.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000002.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000003.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000004.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000006.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000008.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000010.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000011.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000013.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000167.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000168.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000172.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000175.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000176.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000178.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000179.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000181.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000182.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000333.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000335.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000339.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000341.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000342.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000345.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000346.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000348.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/000350.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

...

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009220.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009222.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009223.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009225.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009226.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009228.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009229.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009231.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009232.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009383.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009384.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009385.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009387.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009390.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009391.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009395.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009396.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009397.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009569.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009570.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009572.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009574.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009575.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009578.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009581.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009582.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009583.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009739.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009740.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009741.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009742.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009744.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009750.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009751.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009752.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009753.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009907.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009909.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009910.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009912.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009914.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009915.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009916.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009919.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/009921.jpg
/network/home/bouklihg/Desktop/Thales/Fewshot_Detection/data/VOCdevkit/VOC2007/JPEGImages/0

In [ ]:
# --------------------------------------------------------
# Fast/er R-CNN
# Licensed under The MIT License [see LICENSE for details]
# Written by Bharath Hariharan
# --------------------------------------------------------

import xml.etree.ElementTree as ET
import os,sys
import cPickle
import numpy as np
import argparse
from os import path
from termcolor import colored
import pdb

def get_id(s):
    ss = s.split('_')
    for s in ss:
        if 'novel' in s:
            return s.replace('novel', '')
    return None

def get_novels(root, id=None):
    if root.endswith('txt'):
        if id == 'None':
            return []
        with open(root, 'r') as f:
            novels = f.readlines()
        return novels[int(id)].strip().split(',')
    else:
        return root.split(',')


def filter(detlines, clsfile):
    # pdb.set_trace()
    with open(clsfile, 'r') as f:
        imgids = [l.split()[0] for l in f.readlines() if l.split()[1] == '1']
    dls = [dl for dl in detlines if dl[0] in imgids]

    # dls = [dl for dl in dls if float(dl[1]) > 0.05]
    return dls


def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['pose'] = obj.find('pose').text
        obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)

    return objects

def voc_ap(rec, prec, use_07_metric=False):
    """ ap = voc_ap(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             cachedir,
             ovthresh=0.5,
             use_07_metric=False):
    """rec, prec, ap = voc_eval(detpath,
                                annopath,
                                imagesetfile,
                                classname,
                                [ovthresh],
                                [use_07_metric])

    Top level function that does the PASCAL VOC evaluation.

    detpath: Path to detections
        detpath.format(classname) should produce the detection results file.
    annopath: Path to annotations
        annopath.format(imagename) should be the xml annotations file.
    imagesetfile: Text file containing the list of images, one image per line.
    classname: Category name (duh)
    cachedir: Directory for caching the annotations
    [ovthresh]: Overlap threshold (default = 0.5)
    [use_07_metric]: Whether to use VOC07's 11 point AP computation
        (default False)
    """
    # assumes detections are in detpath.format(classname)
    # assumes annotations are in annopath.format(imagename)
    # assumes imagesetfile is a text file with each line an image name
    # cachedir caches the annotations in a pickle file

    # first load gt
    
    cachefile = os.path.join(cachedir, 'annots.pkl')
    # read list of images
    with open(imagesetfile, 'r') as f:
        lines = f.readlines()
    imagenames = [x.strip() for x in lines]

    if not os.path.isfile(cachefile):
        # load annots
        recs = {}
        for i, imagename in enumerate(imagenames):
            recs[imagename] = parse_rec(annopath.format(imagename))
            if i % 100 == 0:
                print("Reading annotation for {:d}/{:d}".format(i + 1, len(imagenames)))
        # save
        print("Saving cached annotations to {:s}".format(cachefile))
        with open(cachefile, 'w') as f:
            cPickle.dump(recs, f)
    else:
        # load
        with open(cachefile, 'r') as f:
            recs = cPickle.load(f)

    # extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'] == classname]
        bbox = np.array([x['bbox'] for x in R])
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        det = [False] * len(R)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox,
                                 'difficult': difficult,
                                 'det': det}

    # read dets
    detfile = detpath.format(classname)
    with open(detfile, 'r') as f:
        lines = f.readlines()

    # pdb.set_trace()
    clsfile = path.join(path.dirname(imagesetfile), '{}_test.txt')
    clsfile = clsfile.format(classname)
    splitlines = [x.strip().split(' ') for x in lines]
    # print('before', len(splitlines))
    if True:#args.single:
        print('before', len(splitlines))
        splitlines = filter(splitlines, clsfile)
        print('after', len(splitlines))
    # splitlines = bbox_filter(splitlines, conf=0.02)
    # print('after', len(splitlines))
    image_ids = [x[0] for x in splitlines]
    confidence = np.array([float(x[1]) for x in splitlines])
    BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :] if len(BB) != 0 else BB
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    for d in range(nd):
        R = class_recs[image_ids[d]]
        bb = BB[d, :].astype(float)
        ovmax = -np.inf
        BBGT = R['bbox'].astype(float)

        if BBGT.size > 0:
            # compute overlaps
            # intersection
            ixmin = np.maximum(BBGT[:, 0], bb[0])
            iymin = np.maximum(BBGT[:, 1], bb[1])
            ixmax = np.minimum(BBGT[:, 2], bb[2])
            iymax = np.minimum(BBGT[:, 3], bb[3])
            iw = np.maximum(ixmax - ixmin + 1., 0.)
            ih = np.maximum(iymax - iymin + 1., 0.)
            inters = iw * ih

            # union
            uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
                   (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                   (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

            overlaps = inters / uni
            ovmax = np.max(overlaps)
            jmax = np.argmax(overlaps)

        if ovmax > ovthresh:
            if not R['difficult'][jmax]:
                if not R['det'][jmax]:
                    tp[d] = 1.
                    R['det'][jmax] = 1
                else:
                    fp[d] = 1.
        else:
            fp[d] = 1.

    # compute precision recall
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp / float(npos)
    # avoid divide by zero in case the first detection matches a difficult
    # ground truth
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = voc_ap(rec, prec, use_07_metric)

    return rec, prec, ap
    


def _do_python_eval(res_prefix, novel=False, output_dir = 'output'):
    # _devkit_path = '/data2/bykang/pytorch-yolo2/VOCdevkit'
    _devkit_path = 'data/VOCdevkit'
    _year = '2007'
    _classes = ('__background__', # always index 0
        'aeroplane', 'bicycle', 'bird', 'boat',
        'bottle', 'bus', 'car', 'cat', 'chair',
        'cow', 'diningtable', 'dog', 'horse',
        'motorbike', 'person', 'pottedplant',
        'sheep', 'sofa', 'train', 'tvmonitor') 
    _novel_file = 'data/voc_novels.txt'
    novelid = get_id(res_prefix.split('/')[-3])
    print(novelid)
    _novel_classes = get_novels(_novel_file, novelid)

    # _novel_classes = ('bird', 'bus', 'cow', 'motorbike', 'sofa')

    #filename = '/data/hongji/darknet/results/comp4_det_test_{:s}.txt' 
    filename = res_prefix + '{:s}.txt'
    annopath = os.path.join(
        _devkit_path,
        'VOC' + _year,
        'Annotations',
        '{:s}.xml')
    imagesetfile = os.path.join(
        _devkit_path,
        'VOC' + _year,
        'ImageSets',
        'Main',
        'test.txt')
    cachedir = os.path.join(_devkit_path, 'annotations_cache')
    aps = []
    novel_aps = []
    base_aps = []
    # The PASCAL VOC metric changed in 2010
    use_07_metric = True if int(_year) < 2010 else False
    print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
    for i, cls in enumerate(_classes):
        if cls == '__background__':
            continue
        
        rec, prec, ap = voc_eval(
            filename, annopath, imagesetfile, cls, cachedir, ovthresh=0.5,
            use_07_metric=use_07_metric)
        aps += [ap]
        msg = 'AP for {} = {:.4f}'.format(cls, ap)
        # print(rec, prec)
        # msg = 'AP for {} = {:.4f}, recall: {:.4f}, precision: {:.4f}'.format(cls, ap, rec, prec)
        if novel and cls in _novel_classes:
            msg = colored(msg, 'green')
            novel_aps.append(ap)
        else:
            base_aps.append(ap)

        print(msg)
        # print(rec)
        # print(prec)

        with open(os.path.join(output_dir, cls + '_pr.pkl'), 'w') as f:
            cPickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
    print('~~~~~~~~')
    print('Mean AP = {:.4f}'.format(np.mean(aps)))
    if novel:
        print(colored('Mean Base AP = {:.4f}'.format(np.mean(base_aps)), 'green'))
        print(colored('Mean Novel AP = {:.4f}'.format(np.mean(novel_aps)), 'green'))
    print('~~~~~~~~')
    print('Results:')
    # pdb.set_trace()
    for ap in aps:
        print('{:.3f}'.format(ap))
    print('{:.3f}'.format(np.mean(aps)))
    print('~~~~~~~~')
    print('')
    s = ('{:.2f}\t'*20).format(*(np.array(aps) * 100).tolist())
    if novel:
        s += (('{:.2f}\t'*3)).format(np.mean(aps)*100, np.mean(base_aps)*100, np.mean(novel_aps)*100)
    # print(('{:.2f}\t'*20).format(*(np.array(aps) * 100).tolist()))
    print(s)
    print('--------------------------------------------------------------')
    print('Results computed with the **unofficial** Python eval code.')
    print('Results should be very close to the official MATLAB eval code.')
    print('Recompute with `./tools/reval.py --matlab ...` for your paper.')
    print('-- Thanks, The Management')
    print('--------------------------------------------------------------')




_do_python_eval("results/metatunetest1_novel0_neg0/ene000010/comp4_det_test_", novel=True, output_dir = 'output')



<img src="Screenshot from 2020-02-09 11-20-35.png">

The table above shows that when evaluation using three different base/novel splits (novel sets), the proposed model outperforms other few shot object detection learning baselines since the obtained mAP is significantly raised up when considering the proposed method. Moreover, the proposed model converges significantly faster than baseline YOLO detector as shown in the figure bellow.

<img src="Screenshot from 2020-02-09 11-14-08.png">

### Conclusion 

In this notebook, We have presented a novel few-shot object detection learning method that aims at learning using few examples/bounding boxes. 
The proposed method introduces a new model that learns basic features and uses them to detect new classes with few examples. To do so, the method is built on two main components: Feature extractor that extract meta features, and reweighting module used to decide which meta feature is responsible to detect which object.

The introduced method shows better performance then other state-of-the-art few-shot object detection learning methods such as Low-Shot Transfer Detector on challenging object detection datasets such as PASCAL VOC and COCO. On another hand, it also shows that it converges more faster than other counterparts.  

Due to its performance, speed and the ease to understand and implement, this method may represent a relevant solution to tackle the few-shot object detection problem.